In [1]:
import pathlib
import random
from functools import reduce
from collections import defaultdict
import json
import pandas as pd
import numpy as np
from numpy import random

import matplotlib.pyplot as plt
import seaborn as sns

# 공간연산한 결과 데이터 표현하기 위한 패키지 
import folium
from folium import plugins
import shapely
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from IPython.display import display
from tqdm.notebook import tqdm
import sklearn.cluster
import tensorflow as tf
from geoband import API # API 부르기 
import pydeck as pdk
import os
from tqdm import notebook
import tqdm

import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Using Python-MIP package version 1.5.3


### 결과보고서 같이 코드 참고

## data load
- data_list.csv : 제공 데이터를 목록화 한 datframe
- file의 위치에 맞게 데이터를 이동

In [2]:
# 노트북에서 만든 data_list load
file_df = pd.read_csv("./data/data_list.csv")

# order_num type change int to str
file_df["order_num"] = file_df["order_num"].apply(lambda x: str(x))

# # data download
# for i in range(len(file_df)):
#     basic_code = 'SBJ_2102_002'
#     num = i + 1
#     file_name = ".".join(file_df.iloc[i, :])
#     GetCompasData(basic_code, str(num), file_name)

In [3]:
# 동적변수 데이터 할당
# load한 데이터를 : df_1, df_2 ...., df_32로 할당
for idx in tqdm.tqdm(range(len(file_df) - 2)):
#     print(idx)
    file_name = ".".join(file_df.iloc[idx, :])
    globals()[f"df_{idx + 1}"] = gpd.read_file("./data/" + file_name)

100%|██████████| 32/32 [08:11<00:00, 15.36s/it]


### df_500 : 기준 ID
- 정류장 ID와 정류소 ID를 비교를 통해 기준이 되는 ID 선택

In [17]:
# 기준 ID 선정
bus_bay = df_1["정류장ID"].unique()
bus_stop = df_7["정류소ID"].unique()

bay_not_in_stop = []
stop_not_in_bay = []

for bay in bus_bay:
    if bay not in bus_stop:
        bay_not_in_stop.append(bay)
        
for stop in bus_stop:
    if stop not in bus_bay:
        stop_not_in_bay.append(stop)
        
print(f"정류소에 속하지 않은 정류장 수 : {len(bay_not_in_stop)}, 정류장에 속하지 않은 정류소 수 : {len(stop_not_in_bay)}")
# 정류소에 속하지 않은 정류장 수가 많으므로 정류장ID를 기준으로 위도(latitude) 경도(longitude) 부여

정류소에 속하지 않은 정류장 수 : 51, 정류장에 속하지 않은 정류소 수 : 0


In [18]:
# 승하차이력 데이터 정류소 중복 확인
# why? df과 서로 중복이 되는 정류소 ID를 확인하기 위해

stop_id_1 = df_2["정류소ID"].unique()
stop_id_2 = df_3["정류소ID"].unique()
stop_id_3 = df_4["정류소ID"].unique()
stop_id_4 = df_5["정류소ID"].unique()
stop_id_5 = df_6["정류소ID"].unique()
lst = [stop_id_1, stop_id_2, stop_id_3, stop_id_4, stop_id_5]

# 버스정류장별_승하차이력 별 속하지 않는 정류소id 확인
def check_duplicate(x):
    
    check_1 = []
    check_2 = []
    check_3 = []
    check_4 = []
    check_5 = []

    for id in x:
        if id not in stop_id_1:
            check_1.append(id)
        if id not in stop_id_2:
            check_2.append(id)
        if id not in stop_id_3:
            check_3.append(id)
        if id not in stop_id_4:
            check_4.append(id)
        if id not in stop_id_5:
            check_5.append(id)

    if len(check_1) > 0:
        print("승하차이력(1)에 속하지 않는 ID가 있다.", len(check_1))
    else:
        print("기준 이력은 (1)", len(x))
    if len(check_2) > 0:
        print("승하차이력(2)에 속하지 않는 ID가 있다.", len(check_2))
    else:
        print("기준 이력은 (2)", len(x))
    if len(check_3) > 0:
        print("승하차이력(3)에 속하지 않는 ID가 있다.", len(check_3))
    else:
        print("기준 이력은 (3)", len(x))
    if len(check_4) > 0:
        print("승하차이력(4)에 속하지 않는 ID가 있다.", len(check_4))
    else:
        print("기준 이력은 (4)", len(x))
    if len(check_5) > 0:
        print("승하차이력(5)에 속하지 않는 ID가 있다.", len(check_5), "\n")
    else:
        print("기준 이력은 (5)\n", len(x))
        
for x in lst:
    check_duplicate(x)
    
# 승하차이력마다 겹치는 정류소ID가 있으므로 df_1의 정류장ID를 기준으로 합한다.

기준 이력은 (1) 468
승하차이력(2)에 속하지 않는 ID가 있다. 133
승하차이력(3)에 속하지 않는 ID가 있다. 145
승하차이력(4)에 속하지 않는 ID가 있다. 107
승하차이력(5)에 속하지 않는 ID가 있다. 186 

승하차이력(1)에 속하지 않는 ID가 있다. 214
기준 이력은 (2) 549
승하차이력(3)에 속하지 않는 ID가 있다. 185
승하차이력(4)에 속하지 않는 ID가 있다. 124
승하차이력(5)에 속하지 않는 ID가 있다. 261 

승하차이력(1)에 속하지 않는 ID가 있다. 439
승하차이력(2)에 속하지 않는 ID가 있다. 398
기준 이력은 (3) 762
승하차이력(4)에 속하지 않는 ID가 있다. 238
승하차이력(5)에 속하지 않는 ID가 있다. 377 

승하차이력(1)에 속하지 않는 ID가 있다. 389
승하차이력(2)에 속하지 않는 ID가 있다. 325
승하차이력(3)에 속하지 않는 ID가 있다. 226
기준 이력은 (4) 750
승하차이력(5)에 속하지 않는 ID가 있다. 312 

승하차이력(1)에 속하지 않는 ID가 있다. 251
승하차이력(2)에 속하지 않는 ID가 있다. 245
승하차이력(3)에 속하지 않는 ID가 있다. 148
승하차이력(4)에 속하지 않는 ID가 있다. 95
기준 이력은 (5)
 533


### df_500 : 훈련 데이터 셋 기준 df 추출

In [19]:
# 승하차 관련 변수 추가
get_in_out_lst = df_2.iloc[0, 6:-1].index.values

for col in get_in_out_lst:
    df_1[col] = 0

In [20]:
# 승하차 관련 변수 타입 변경
def change_dtype(df):
    for col in get_in_out_lst:
        df[col] = df[col].astype(int)
        
df_lst = [df_2, df_3, df_4, df_5, df_6]
for df in df_lst:
    change_dtype(df)

In [21]:
# 승하차 변수 더하기
def add_get_in_out(criterion, add_df):
    for idx in tqdm.tqdm(range(len(criterion))):
        id = criterion.iloc[idx, 0]
        
        check_index = add_df.index.values
        if id in check_index:
            for col in get_in_out_lst:
                criterion.loc[idx, col] += add_df.loc[id, col]
                    
                    
for add_df in df_lst:
    add_df = add_df.groupby("정류소ID")[get_in_out_lst].agg("sum")
    add_get_in_out(df_1, add_df)

100%|██████████| 1179/1179 [00:01<00:00, 725.27it/s]


In [23]:
# 500m 기준 TOD 변수들을 뽑기 위해 point 기준 500m 버퍼 생성

# point 생성
# geopandas의 gis 관련 함수를 사용하기 위해서는 기준이 되는 point, polygon과 같은
# geometry타입의 변수가 있어야 된다.
geometry = gpd.points_from_xy(df_1.lon, df_1.lat)
# point 할당
df_1["geometry"] = geometry
# 1m
coor_1m = (1/88.74/1000)
#500m 버퍼 생성
df_1["buffer"] = df_1["geometry"].apply(lambda x: x.buffer(coor_1m*500))

df_1.to_csv("수원시_버스정류장_승하차이력.csv", index = False)

### 지하철 역 수 도출을 위한 df

In [ ]:
# 변수 추출
subway_col = df_9.iloc[0, 5:-1].index.values

# , 제거 후 int형으로 변경
def remove(x):
    x = x.replace(",", "")
    return x

for i in subway_col:
    df_9[i] = df_9[i].apply(lambda x: remove(x))
    df_9[i] = df_9[i].astype(int)
    
subway_sum_df = df_9.groupby(["역명", "승하차"])[subway_col].agg("sum").reset_index()

subway_sum_df["lon"] = 0
subway_sum_df["lat"] = 0

In [29]:
# 역별 경도 위도 부여
for idx in range(len(subway_sum_df)):
    subway_id = subway_sum_df.iloc[idx, 0]
    try:
        subway_sum_df.loc[idx, "lon"] = df_8[df_8["역사명"] == subway_id]["lon"].values
        subway_sum_df.loc[idx, "lat"] = df_8[df_8["역사명"] == subway_id]["lat"].values
    except Exception as e:
        print(subway_id)
        
# 기준 id가 달라서 부여 되지 않은 경도 위도 추출 후 부여
lon = df_8[df_8["역사명"] == "수원역"]["lon"].values
lat = df_8[df_8["역사명"] == "수원역"]["lat"].values

subway_sum_df.loc[10, "lon"] = lon
subway_sum_df.loc[10, "lat"] = lat
subway_sum_df.loc[11, "lon"] = lon
subway_sum_df.loc[11, "lat"] = lat

# geometry point 생성
geometry = gpd.points_from_xy(subway_sum_df.lon, subway_sum_df.lat)
subway_sum_df["geometry"] = geometry


subway_sum_df.to_csv("수원시_지하철역별_이용현황.csv", index = False)

수원
수원


### 유동인구 - 인구정보 결합데이터 추출
- 유동인구 병합데이터 추출

In [30]:
# 계산을 위해 dtype 변환
df_14.iloc[:, 1:-3] = df_14.iloc[:, 1:-3].astype(float)
df_15.iloc[:, 1:-3] = df_15.iloc[:, 1:-3].astype(float)
df_16.iloc[:, 1:-3] = df_16.iloc[:, 1:-3].astype(float)

# 경도 위도 데이터 타입 변환 why? string 형태에서 1480 과 148을 다른것으로 인식하기 때문
def dtype_change(x):
    x["lon"] = x["lon"].astype(float)
    x["lat"] = x["lat"].astype(float)
    
pop_lst = [df_14, df_15, df_16]

for df in pop_lst:
    dtype_change(df)

In [31]:
# 유동인구 변수를 추출하기 위해 버스 이용가능 시간인 06 ~ 23시까지 indexing하고 합한 뒤
df_14["sum_pop"] = df_14.loc[:, "TMST_06":"TMST_23"].agg("sum", axis = 1).values

# 1 ~ 12월의 평균값 도출
df_pop = df_14.groupby(["lon", "lat"])["sum_pop"].agg(['sum',
                                                       "mean"]).rename({"mean":"pop"}, axis = 1).reset_index()

In [14]:
df_pop = df_pop[["pop", "lon", "lat"]]
df_pop.head()

,pop,lon,lat
0,0.0350,126.927348,37.278703
1,3.2810,126.927389,37.273295
2,4.8575,126.927392,37.272844
3,1.1420,126.927395,37.272393
4,2.0870,126.927399,37.271943


In [ ]:
# 필요 변수 추출
# 광고 Targets에 할용 될 변수
col_15 = df_15.iloc[:, 1:-3].columns.values

# 경도 위도 기준 변수 별 평균 도출
# 병합에 사용될 df 
df_15_crit = df_15.groupby(["lon", "lat"])[col_15].agg("mean").reset_index()

# 참고할 df 추출
col_16 = df_16.iloc[:, 1:-3].columns.values
df_16_crit = df_16.groupby(["lon", "lat"])[col_16].agg("mean").reset_index()

In [17]:
# 유동인구 데이터들은 수원시를 50m x 50m격자를 기준으로 제공 되기 때문에
# 기준 경도 위도 추출을 통해 데이터 병합에 사용
crit_pop = df_pop[["lon", "lat"]]
crit_15 = df_15_crit[["lon", "lat"]]
crit_16 = df_16_crit[["lon", "lat"]]
df_new_2 = pd.concat([crit_pop, crit_15, crit_16]).reset_index(drop = True)

# groupby에 사용할 임시 변수 추가
df_new_2["pop"] = 0

# 기준 경도 위도 추출
df_new_2 = df_new_2.groupby(["lon", "lat"])["pop"].agg(["count", "mean"]).reset_index()[["lon", "lat"]]

# 필요 변수 추가
df_new_2["pop"] = 0
df_new_2[col_15] = 0
df_new_2[col_16] = 0

In [ ]:
# 기준 경도 위도에 데이터 할당
# try: except를 예외처리를 한 이유?
# 기준이 되는 위도 경도를 뽑았지만 제공되는 유동인구 데이터 별 
# 제공하는 데이터 포인트가 다르기 때문에 기준이 되는 위도 경도에 해당하는 데이터가
# 결측치일 때 오류를 방지하고 for문을 돌리기 위해
for idx in tqdm.tqdm(range(len(df_new_2))):
    try:
        pop = df_pop[(df_pop["lon"] == df_new_2.loc[idx, "lon"]) &
                (df_pop["lat"] == df_new_2.loc[idx, "lat"])]["pop"].values
        df_new_2.loc[idx, "pop"] = pop
        
    except Exception as e:
        print("시간대별 유동인구 오류 :", idx, e)
        
    try:
        sex_pop = df_15_crit[(df_15_crit["lon"] == df_new_2.loc[idx, "lon"]) &
                (df_15_crit["lat"] == df_new_2.loc[idx, "lat"])][col_15].values[0]
        df_new_2.loc[idx, col_15] = sex_pop
    except Exception as e:
        print("성연령별 유동인구 오류 :", idx, e)

    try:
        day_pop = df_16_crit[(df_16_crit["lon"] == df_new_2.loc[idx, "lon"]) &
                (df_16_crit["lat"] == df_new_2.loc[idx, "lat"])][col_16].values[0]
        df_new_2.loc[idx, col_16] = day_pop
    except Exception as e:
        print("요일별 유동인구 오류 :", idx, e)

In [19]:
# 유동인구 데이터 geometry생성
# geopandas 함수 사용을 위해
def make_geometry_point(df):
    geometry = gpd.points_from_xy(df.lon, df.lat)
    df["geometry"] = geometry    

make_geometry_point(df_new_2)

- 인구정보 병합데이터 추출

In [39]:
# 인구정보 결측치 채우기
df_17.fillna(0, inplace = True)
df_18.fillna(0, inplace = True)
df_19.fillna(0, inplace = True)
df_28.fillna(0, inplace = True)

old_val = df_17["val"]
worker_val = df_18["val"]
youth_val = df_19["val"]
building_area = df_28["val"]

# 기준이 되는 인구정보 df
df_new = pd.DataFrame({"gid":df_17["gid"], "old":old_val,
                      "worker":worker_val, "youth":youth_val, "building_area":building_area
                       ,"geometry":df_17["geometry"]})

df_new.to_csv("격자_데이터.csv", index = False)

In [23]:
# sjoin(spatial join을 위해 ) df -> geodf
# 인구정보 df
df_new = gpd.GeoDataFrame(df_new)
# 유동인구 df
crit = gpd.GeoDataFrame(df_new_2)

# sjoin을 통합 병합
check_df = gpd.sjoin(df_new, crit, op = "contains", how = "left")

# 구해야 되는 변수 
col_human = df_new.iloc[0, 1:-1].index.values
col_pop = crit.iloc[0, 2:-1].index.values

# 인구정보 : 평균 (result :이전 값 그대로)
human_df = check_df.groupby("gid")[col_human].agg("mean").reset_index()
# 유동인구 : 합계 (gid 격자안에 속하는 값 모두 더하기)
pop_df = check_df.groupby("gid")[col_pop].agg("sum").reset_index()

# gid 기준으로 병합
final_df = pd.merge(human_df, pop_df, on="gid")

# 기존 폴리곤 부여(why? 시각화와 sjoin에 쓰기 위해)
final_df["geometry"] = df_new["geometry"]

final_df.to_csv("유동인구_인구정보_결합.csv", index = False)

### 도로 데이터 병합

In [ ]:
# full time 제외 버스 운영 시간으로 필터링
traffic_crit_df = df_23[df_23["시간적범위"] != "fulltime"].reset_index(drop = True)
# 계산 하기 위해 dtype 변경
traffic_crit_df.iloc[:, 7:-1] = traffic_crit_df.iloc[:, 7:-1].astype(int)
# 버스 운행 시간 필터링
traffic_crit_df = traffic_crit_df[(traffic_crit_df["시간적범위"] >= 6)].reset_index(drop = True, inplace = True)

# 사용할 변수 추출
traffic_col = traffic_crit_df.iloc[:, 8:-1].columns.values
# linkid기준 그룹핑 후 변수로 합계 추출
traffic_df = traffic_crit_df.groupby("상세도로망_LinkID")[traffic_col].agg("sum").reset_index()
# 기준 df 추출
traffic_crit_df_subset = traffic_crit_df.drop_duplicates(subset = "상세도로망_LinkID", keep = "last").reset_index(drop = True).iloc[:, :7]
# 병합 주의사항) 전체_추정교통량은 승용차 + 버스 + 화물차랑 약간씩 다른 경우가 있다.
df_new_3 = pd.merge(traffic_df, traffic_crit_df_subset, on = "상세도로망_LinkID")

In [25]:
# df_23 24 병합
df_24_25 = pd.merge(df_24, df_25.iloc[:, [0, -2]], on = "상세도로망_LinkID")
# 불필요 변수 제거
df_24_25 = df_24_25.drop("geometry", axis = 1)
# 필수 df slicing
df_24_25_subset = df_24_25.iloc[:, [0,-1,-2]]
# df_23_24, df_new_3 병합 why left? df_new_3이 데이터 포인트가 많기 때문
df_new_3_crit = pd.merge(df_new_3, df_24_25_subset, on = "상세도로망_LinkID", how = "left")

re_col = ['상세도로망_LinkID', '도로등급', '링크길이', '도로명', '시도명', '시군구명', '읍면동명', '전체_추정교통량', '승용차_추정교통량',
          '버스_추정교통량', '화물차_추정교통량', '혼잡시간강도', '혼잡빈도강도']

# df_new_3_crit 순서 재정렬
df_new_3_crit = df_new_3_crit[re_col]

road_geo = df_22.iloc[:, [0, -1]]
df_new_3_crit["link_id"] = df_new_3_crit["상세도로망_LinkID"].apply(lambda x: x[:-2])

# 도로 데이터 병합
road_transport_geo = pd.merge(df_new_3_crit, road_geo, how = "left")
road_transport_geo.to_csv("./data/road_transport.csv", index = False)

### 토지이용특성 변수 도출
- 제공하는 코드명에 직접 설정한 변수명 설정

In [30]:
# 건물 데이터 중 slicing
build_df = df_27.iloc[:, [0,-1]]
# dtype 변경 why? bhilding_code랑 비교 하기 위해
build_df["BDTYP_CD"] = build_df["BDTYP_CD"].astype(float)

# 코드정의서 중 건물용도코드 부분에 직접 변수명 설정
building_code = pd.read_csv("./data/건물용도코드_확인_필요.csv")
building_code.dropna(axis = 0, inplace = True)

build_df["purpose"] = 0

for idx in range(len(building_code)):
    try:
        index = build_df[build_df["BDTYP_CD"] == building_code.iloc[idx, 0]].index.values
        build_df.iloc[index, -1] = building_code.iloc[idx, 1]
    except:
        print(idx)
        
        
# 결측치 채우기
fill_col = build_df[build_df["purpose"] == 0]["BDTYP_CD"].unique()
fill_col.sort()

fill_lst = ["문화체육시설", "공공시설", "공공시설", "공공시설", "공공시설", "공공시설", "공공시설", "공공시설",
      "공공시설", "공공시설", "공공시설", "상업시설", "공공시설", "상업시설", "상업시설", "문화체육시설",
      "상업시설", "상업시설", "문화체육시설", "상업시설"]

fill_df = pd.DataFrame({"코드":fill_col, "변수":fill_lst})
new_building_code = pd.concat([building_code, fill_df]).reset_index(drop = True)


for idx in range(len(new_building_code)):
    try:
        index = build_df[build_df["BDTYP_CD"] == new_building_code.iloc[idx, 0]].index.values
        build_df.iloc[index, -1] = new_building_code.iloc[idx, 1]
    except:
        print(idx)

## 기준 df
- df_500 : 버스정류장 위치 기준 반경 500m의 버퍼를 통해 TOD 변수들을 뽑은 훈련 데이터 셋
- df_100 : 100m 격자인 gid를 기준으로 TOD 변수들은 뽑은 데이터로서 수원시의 특성을 파악하기 위해 사용
- df_100_to_500 : 100m 격자의 중심점 기준 반경 500m의 버퍼를 통해 TOD 변수들을 뽑은 테스트 데이터 셋으로서 실제 입지선정에 사용

### df_500 데이터 추출
- 버스정류장 별 반경 500m 버펑 그리기

In [37]:
# intersection을 위해 buffer slicing 후 geometry로 컬럼명 변경
df_1_area = df_1.loc[:, ["정류장ID", "buffer"]].rename({"buffer":"geometry"}, axis = 1)

# df_1_area를 기준으로 buil_df intersection
bus_with_build_area_df = gpd.overlay(df_1_area, build_df, how = "intersection")

# 건물면적 구하기
bus_with_build_area_df["area"] = bus_with_build_area_df.area

# 정류장 ID와 건물분류별 면적 합하기
build_area = bus_with_build_area_df.groupby(["정류장ID", "purpose"])["area"].agg("sum").reset_index()

# pivot table 활용하여 정류장ID별 건물불류 변수 table 생성
table = pd.pivot_table(build_area, values = "area", index = "정류장ID", columns = "purpose").reset_index()

# 결측치 처리
table = table.fillna(0)

# # 500m 버퍼 내의 면적을 구하기 위해 500m 버퍼 면적 구하기(면적)
# denominator = df_1_area[df_1_area["정류장ID"] == '201000266'].area.values[0]

# #  intersection / 500m 면적
# # table.iloc[:, 1:] = table.iloc[:, 1:].apply(lambda x: x / denominator)

# 버퍼 내 전체 건물 면적 구하기
table["건물면적"] = table.iloc[:, 1:].sum(axis = 1)

# 서버가 끊기는 것을 고려하여 저장하여 사용
bus_with_build_area_df.to_csv("./data/500_build_polygon.csv", index = False)

# 500m 버스정류장, area 병합
# 토지이용특성을 부여할 기준 df
df_1_build = pd.merge(df_1, table, how = "left")

df_1_build.to_csv("정류장_빌딩_병합.csv", index = False)

- 토지용특성 변수 추출(인구사회특성의 주거 연면적도 같이 추출)

In [122]:
df_1_build_crit = df_1_build.loc[:, ["정류장ID", "buffer"]].rename({"buffer":"geometry"}, axis = 1)

# df_1_build_pop sjoin(default : "intersection")
df_1_build_pop_final_inter = gpd.overlay(df_1_build_crit, final_df, how = "intersection")

# 기준 면적(100mx100m 격자)
crit_area = final_df["geometry"].area

# 가중치 구하기
df_1_build_pop_final_inter["coef"] = df_1_build_pop_final_inter["geometry"].area / crit_area


# 합계 필요한 변수 추출
df_1_build_pop_final_inter_col = df_1_build_pop_final_inter.iloc[:, 2:-2].columns.values

# 가중치 부여
for idx in range(len(df_1_build_pop_final_inter_col)):
    df_1_build_pop_final_inter[df_1_build_pop_final_inter_col[idx]] = df_1_build_pop_final_inter[df_1_build_pop_final_inter_col[idx]].mul(df_1_build_pop_final_inter["coef"])

# 정류장ID 기준으로 합계
df_1_buil_pop_final_group = df_1_build_pop_final_inter.groupby("정류장ID")[df_1_build_pop_final_inter_col].agg('sum').reset_index()

# 병합
df_1_build_pop = pd.merge(df_1_build, df_1_buil_pop_final_group, on = "정류장ID")

- 지하철역 수

In [41]:
geometry = gpd.points_from_xy(df_8.lon, df_8.lat)
df_8["geometry"] = geometry

# 지하철역 수
ex = gpd.sjoin(df_1_build_pop.loc[:, ["정류장ID","buffer"]].rename({"buffer":"geometry"}, axis = 1), df_8, how = 'inner', op = "contains")
# 지하철역 수 count
ex = ex["정류장ID"].value_counts().reset_index().rename({"index":"정류장ID", "정류장ID":"지하철역수"}, axis = 1)

# df_1_build_pop에 subway 병합
df_1_build_pop_subway = pd.merge(df_1_build_pop, ex, how = "left")

- 버스정류장 수

In [125]:
# 버스정류장
# df_1_build_pop(버스정류장 버퍼)
bus_buffer = df_1_build_pop

# 버스정류장 point
bus_point = df_1_build_pop_subway.loc[:, ["정류장ID", "geometry"]]

# 버스정류장 수 구하기
bus_count = gpd.sjoin(bus_buffer, bus_point, "inner", op = "contains")["정류장ID_left"].value_counts()

# 버스정류장 칼럼 명 변경
bus_count = bus_count.reset_index().rename({"index":"정류장ID", "정류장ID_left":"버스정류장수"}, axis = 1)

# 병합
df_1_build_pop_subway_bus = pd.merge(df_1_build_pop_subway, bus_count)

- 주차장 수

In [126]:
# 주차장
df_12.drop("geometry", axis = 1, inplace = True)

# 주차장 point 구하기
geometry = gpd.points_from_xy(df_12.lon, df_12.lat)
df_12["geometry"] = geometry

# 주차장수 구하기
park_lot = gpd.sjoin(df_1_build_pop_subway_bus.loc[:, ["정류장ID", "buffer"]].rename({"buffer":"geometry"}, axis = 1), df_12, op = "contains")["정류장ID"].value_counts()
park_lot = park_lot.reset_index().rename({"index":"정류장ID", "정류장ID":"주차장수"}, axis = 1)

# 주차장 수 병합
df_1_build_pop_subway_bus_park = pd.merge(df_1_build_pop_subway_bus, park_lot, how = "left")

- 접근성(거리) : 버스정류장 기준점과 각 건물의 중심점의 직선거리

In [128]:
# 접근성
center = bus_with_build_area_df["geometry"].centroid
bus_with_build_area_df["center"] = center
build_df["center"] = build_df["geometry"].centroid
df_1_buffer = df_1.loc[:, ["정류장ID", "buffer","geometry"]].rename({"geometry":"bus_center", "buffer":"geometry"}, axis = 1)

bus_with_build_area_df = gpd.overlay(df_1_buffer, build_df, how = "intersection")
    
    
# 거리 구하기
bus_with_build_area_df["distance"] = 0

dis = bus_with_build_area_df["center"].distance(bus_with_build_area_df["bus_center"])
bus_with_build_area_df["distance"] = dis
    
# 거리 평균
bus_build_distance = bus_with_build_area_df.groupby("정류장ID")["distance"].agg("mean").reset_index()

# 병합
df_1_build_pop_subway_bus_park_acc = pd.merge(df_1_build_pop_subway_bus_park, bus_build_distance, how="left")

# save df
df_1_build_pop_subway_bus_park_acc.to_csv("./data/df_500.csv", index = False)

df_1_build_pop_subway_bus_park_acc.to_csv("./data/df_500_kr.csv", index = False, encoding="euc-kr")

### df_100 수원시 데이터 추출
- 인구정보와 유동인구 변수들은 수원시 데이터를 기준으로 추출 했기 때문에 따로 뽑지 않아도 됨

In [35]:
final_df["gid_center"] = 0
center = final_df["geometry"].centroid
final_df["gid_center"] = center

- 토지이용특성 변수 추출

In [130]:
# 100m 밀도 구하기
gid_build = gpd.overlay(final_df, build_df, how = "intersection")

gid_build["area"] = 0
area = gid_build["geometry"].area
gid_build["area"] = area

gid_build_area = gid_build.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()


gid_build_area_pivot = pd.pivot_table(gid_build_area, values = "area",
                                      columns = "purpose", index = "gid").reset_index().fillna(0)


gid_build_area_pivot["건물면적"] = gid_build_area_pivot.iloc[:, 1:].sum(axis = 1)

# 100m_area
final_df_area = pd.merge(final_df, gid_build_area_pivot, how = "left")

In [66]:
# Server 이용시간을 고려해 저장
bus_buffer.to_csv("./data/bus_buffer.csv", index = False)
bus_point.to_csv("./data/bus_point.csv", index = False)
build_df.to_csv("./data/build_df.csv", index = False)

- 지하철역 수

In [76]:
# 지하철
gid_sub = gpd.sjoin(final_df_area.loc[:, ["gid", "geometry"]], df_8, how = "inner")
gid_sub_count = gid_sub["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"지하철역수"}, axis = 1)
final_df_area_subway = pd.merge(final_df_area, gid_sub_count, how = "left")

- 버스정류장 수

In [78]:
# 버스
gid_bus = gpd.sjoin(final_df_area.loc[:, ["gid", "geometry"]], bus_point, how = "inner").reset_index(drop = True)
gid_bus_count = gid_bus["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"버스정류장수"}, axis = 1)
gid_bus = gpd.sjoin(final_df_area.loc[:, ["gid", "geometry"]], bus_point, how = "inner").reset_index(drop = True)
final_df_area_subway_bus = pd.merge(final_df_area_subway, gid_bus_count, how = "left")

- 주차장 수

In [79]:
# 주차장 df_12
gid_park = gpd.sjoin(final_df_area.loc[:, ["gid", "geometry"]], df_12, how = "inner").reset_index(drop = True)
gid_park_count = gid_park["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"주차장수"}, axis = 1)
final_df_area_subway_bus_park = pd.merge(final_df_area_subway_bus, gid_park_count, how = "left")

- 접근성

In [80]:
# 접근성
gid_build_center = gpd.sjoin(final_df_area.loc[:, ["gid", "geometry", "gid_center"]], build_df, op = "intersects")

gid_build_center["distance"] = 0
gid_build_center.reset_index(drop = True, inplace = True)

distance = gid_build_center["gid_center"].distance(gid_build_center["center"])
gid_build_center["distance"] = distance
    
gid_build_center_dist = gid_build_center.groupby("gid")["distance"].agg("mean").reset_index()
final_df_area_subway_bus_park_acc = pd.merge(final_df_area_subway_bus_park, gid_build_center_dist, how = "left")
final_df_area_subway_bus_park_acc.to_csv("./data/df_100.csv", index = False)

100%|██████████| 95991/95991 [01:25<00:00, 1116.92it/s]


In [ ]:
final_df

### df_100_to_500 훈련데이터 추출
- 100m 기준점 기준 500m 버퍼 그리기

In [ ]:
# server 끊어졌을 때 다시실행
final_df = pd.read_csv("유동인구_인구정보_결합.csv")
df_gid_pop = pd.read_csv("df_gid_pop.csv")
final_df = gpd.GeoDataFrame(final_df)
df_gid_pop = gpd.GeoDataFrame(df_gid_pop)
def str_to_geo(x):
    a = shapely.wkt.loads(x)
    return a

final_df["geometry"] = final_df["geometry"].apply(lambda x: str_to_geo(x))
df_gid_pop["geometry"] = df_gid_pop["geometry"].apply(lambda x: str_to_geo(x))
df_gid_pop["gid_center"] = df_gid_pop["gid_center"].apply(lambda x: str_to_geo(x))
df_gid_pop["gid_center"] = df_gid_pop["gid_center"].astype("geometry")

df_gid_pop_coef = df_gid_pop.loc[:, ["gid_1", "gid_center"]]
final_df["gid_center"] = 0
center = final_df["geometry"].centroid
final_df["gid_center"] = center

gid_with_rink = df_gid_pop_coef["gid_1"].unique()

# gid index화 후 loc로 뽑기
df_gid = final_df.set_index("gid").loc[gid_with_rink, :].reset_index()

# 1m
coor_1m = (1/88.74/1000)

# gid격자 buffer화
df_gid["buffer"] = 0
df_gid_buffer = df_gid["gid_center"].buffer(coor_1m * 500)
df_gid["buffer"] = df_gid_buffer

# df_gid의 기준 추출
df_gid_crit = df_gid.loc[:, ["gid", "buffer"]].rename({"buffer":"geometry"}, axis = 1)

build_df = pd.read_csv("./data/build_df.csv")
build_df = gpd.GeoDataFrame(build_df)
build_df["geometry"] = build_df["geometry"].apply(lambda x: str_to_geo(x))
build_df["center"] = build_df["center"].apply(lambda x: str_to_geo(x))
build_df["center"] = build_df["center"].astype("geometry")
import time

- 버스정류장은 도로가 있는 지역에 설치가 가능하므로 df_100 수원시 데이터와 도로 데이터를 결합하여 추출

In [ ]:
# final_df와 도로 데이터 sjoin
a = gpd.sjoin(final_df, gpd.GeoDataFrame(road_transport_geo), how = "inner")

# sjoin gid 뽑기
gid_with_rink = a["gid"].unique()

# gid index화 후 loc로 뽑기
df_gid = final_df.set_index("gid").loc[gid_with_rink, :].reset_index()

# gid격자 buffer화
df_gid["buffer"] = 0
df_gid_buffer = df_gid["gid_center"].buffer(coor_1m * 500)
df_gid["buffer"] = df_gid_buffer

# df_gid의 기준 추출
df_gid_crit = df_gid.loc[:, ["gid", "buffer"]].rename({"buffer":"geometry"}, axis = 1)

# df_gid overlay pop
df_gid_pop = gpd.overlay(df_gid_crit, final_df, how = "intersection")

# csv 파일 저장
df_gid_pop.to_csv("df_gid_pop.csv", index = False)

- 유동인구- 인구정보 변수

In [11]:
# gid 별 area 구하기
final_df["area"] = final_df["geometry"].area

# coef 변수 생성
df_gid_pop["coef"] = 0

# gid별 실제 area 구하기
a = pd.merge(df_gid_pop.loc[:, ["gid_1", "old"]].rename({"gid_1":"gid"}, axis = 1), 
             final_df.loc[:, ["gid", "area"]], how = "left")

# coef 가중치 구하기
df_gid_pop["coef"] = df_gid_pop["geometry"].area / a["area"]

# 계산 필요 변수 유동인구는 시간대별만 활용
df_gid_pop_col = df_gid_pop.iloc[:, 2:-4].columns.values


# 가중치 부여
for idx in tqdm.tqdm(range(len(df_gid_pop_col))):
    df_gid_pop[df_gid_pop_col[idx]] = df_gid_pop[df_gid_pop_col[idx]].mul(df_gid_pop["coef"])
    
# 저장
df_gid_pop.to_csv("df_gid_pop_coef.csv", index = False)

# 그룹화
df_gid_pop_coef_group = df_gid_pop.groupby("gid_1")[df_gid_pop_col].agg('sum').reset_index().rename({"gid_1":"gid"}, axis = 1)

# 저장
df_gid_pop_coef_group.to_csv("df_gid_pop_coef_group.csv", index = False)

100%|██████████| 24/24 [00:00<00:00, 252.58it/s]


- 지하철역 수

In [37]:
# 지하철
geo = gpd.points_from_xy(df_8.lon, df_8.lat)
df_8["geometry"] = geo

gid_sub = gpd.sjoin(df_gid_crit, df_8, how = "inner")
gid_sub_count = gid_sub["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"지하철역수"}, axis = 1)
df_gid_pop_coef_group_subway = pd.merge(df_gid_pop_coef_group, gid_sub_count, how = "left")

- 버스정류장 수

In [39]:
# 버스
bus_point = gpd.GeoDataFrame(pd.read_csv("./data/bus_point.csv"))
bus_point["geometry"] = bus_point["geometry"].apply(lambda x: str_to_geo(x))

gid_bus = gpd.sjoin(df_gid_crit, bus_point, how = "inner").reset_index(drop = True)
gid_bus_count = gid_bus["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"버스정류장수"}, axis = 1)
df_gid_pop_coef_group_subway_bus = pd.merge(df_gid_pop_coef_group_subway, gid_bus_count, how = "left")

- 지하철역 수

In [43]:
# 주차장 df_12
geo = gpd.points_from_xy(df_12.lon, df_12.lat)
df_12["geometry"] = geo

gid_park = gpd.sjoin(df_gid_crit, df_12, how = "inner").reset_index(drop = True)
gid_park_count = gid_park["gid"].value_counts().reset_index().rename({"index":"gid", "gid":"주차장수"}, axis = 1)
df_gid_pop_coef_group_subway_bus_park = pd.merge(df_gid_pop_coef_group_subway_bus, gid_park_count, how = "left")

- 접근성

In [44]:
# 접근성
df_gid_crit["gid_center"] = df_gid_crit["geometry"].centroid
gid_build_center = gpd.sjoin(df_gid_crit, build_df, op = "intersects")

gid_build_center["distance"] = 0
gid_build_center.reset_index(drop = True, inplace = True)

distance = gid_build_center["gid_center"].distance(gid_build_center["center"])
gid_build_center["distance"] = distance
    
gid_build_center_dist = gid_build_center.groupby("gid")["distance"].agg("mean").reset_index()
df_gid_pop_coef_group_subway_bus_park_acc = pd.merge(df_gid_pop_coef_group_subway_bus_park, gid_build_center_dist, how = "left")
df_gid_pop_coef_group_subway_bus_park_acc.to_csv("./data/df_100_to_500.csv", index = False)

- 토지이용특성 변수 추출
    * 연산이 오래 걸리기 때문에 서버 이용시간을 고려해서 indexing 해서 구함

In [3]:
start = time.time()
gid_buffer_inter_1 = gpd.overlay(df_gid_crit[:1000], build_df, how = "intersection")

gid_buffer_inter_1.to_csv("gid_buffer_inter_1.csv", index = False)
end = time.time()
print("time", end - start)

time 596.1591682434082


In [47]:
# 저장 한거 불러오기
gid_buffer_inter_1 = pd.read_csv("gid_buffer_inter_1.csv")
gid_buffer_inter_1 = gpd.GeoDataFrame(gid_buffer_inter_1)
gid_buffer_inter_1["geometry"] = gid_buffer_inter_1["geometry"].apply(lambda x: str_to_geo(x))

gid_buffer_inter_1["area"] = gid_buffer_inter_1["geometry"].area
gid_area_1 = gid_buffer_inter_1.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()

gid_area_1_pivot = pd.pivot_table(data = gid_area_1, index = "gid", columns="purpose", values = "area")
gid_area_1_pivot = gid_area_1_pivot.reset_index()

In [4]:
start = time.time()
gid_buffer_inter_2 = gpd.overlay(df_gid_crit[1000:2000], build_df, how = "intersection")
gid_buffer_inter_2.to_csv("gid_buffer_inter_2.csv", index = False)
end = time.time()
print("time", end - start)

time 913.8244581222534


In [48]:
gid_buffer_inter_2 = pd.read_csv("gid_buffer_inter_2.csv")
gid_buffer_inter_2 = gpd.GeoDataFrame(gid_buffer_inter_2)
gid_buffer_inter_2["geometry"] = gid_buffer_inter_2["geometry"].apply(lambda x: str_to_geo(x))

gid_buffer_inter_2["area"] = gid_buffer_inter_2["geometry"].area
gid_area_2 = gid_buffer_inter_2.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()

gid_area_2_pivot = pd.pivot_table(data = gid_area_2, index = "gid", columns="purpose", values = "area")
gid_area_2_pivot = gid_area_2_pivot.reset_index()

In [5]:
start = time.time()
gid_buffer_inter_3 = gpd.overlay(df_gid_crit[2000:3000], build_df, how = "intersection")
gid_buffer_inter_3.to_csv("gid_buffer_inter_3.csv", index = False)
end = time.time()
print("time", end - start)

time 2098.75452709198


In [49]:
gid_buffer_inter_3 = pd.read_csv("gid_buffer_inter_3.csv")
gid_buffer_inter_3 = gpd.GeoDataFrame(gid_buffer_inter_3)
gid_buffer_inter_3["geometry"] = gid_buffer_inter_3["geometry"].apply(lambda x: str_to_geo(x))

gid_buffer_inter_3["area"] = gid_buffer_inter_3["geometry"].area
gid_area_3 = gid_buffer_inter_3.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()

gid_area_3_pivot = pd.pivot_table(data = gid_area_3, index = "gid", columns="purpose", values = "area")
gid_area_3_pivot = gid_area_3_pivot.reset_index()

In [6]:
start = time.time()
gid_buffer_inter_4 = gpd.overlay(df_gid_crit[3000:4000], build_df, how = "intersection")
gid_buffer_inter_4.to_csv("gid_buffer_inter_4.csv", index = False)
end = time.time()
print("time", end - start)

time 1640.516138792038


In [50]:
gid_buffer_inter_4 = pd.read_csv("gid_buffer_inter_4.csv")
gid_buffer_inter_4 = gpd.GeoDataFrame(gid_buffer_inter_4)
gid_buffer_inter_4["geometry"] = gid_buffer_inter_4["geometry"].apply(lambda x: str_to_geo(x))

gid_buffer_inter_4["area"] = gid_buffer_inter_4["geometry"].area
gid_area_4 = gid_buffer_inter_4.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()

gid_area_4_pivot = pd.pivot_table(data = gid_area_4, index = "gid", columns="purpose", values = "area")
gid_area_4_pivot = gid_area_4_pivot.reset_index()

In [3]:
start = time.time()
gid_buffer_inter_5 = gpd.overlay(df_gid_crit[4000:], build_df, how = "intersection")
gid_buffer_inter_5.to_csv("gid_buffer_inter_5.csv", index = False)
end = time.time()
print("time", end - start)

time 1008.5128085613251


In [51]:
gid_buffer_inter_5 = pd.read_csv("gid_buffer_inter_5.csv")
gid_buffer_inter_5 = gpd.GeoDataFrame(gid_buffer_inter_5)
gid_buffer_inter_5["geometry"] = gid_buffer_inter_5["geometry"].apply(lambda x: str_to_geo(x))

gid_buffer_inter_5["area"] = gid_buffer_inter_5["geometry"].area
gid_area_5 = gid_buffer_inter_5.groupby(["gid", "purpose"])["area"].agg("sum").reset_index()

gid_area_5_pivot = pd.pivot_table(data = gid_area_5, index = "gid", columns="purpose", values = "area")
gid_area_5_pivot = gid_area_5_pivot.reset_index()

In [52]:
gid_area = pd.concat([gid_area_1_pivot, gid_area_2_pivot, gid_area_3_pivot,
          gid_area_4_pivot, gid_area_5_pivot]).reset_index(drop = True)

In [53]:
gid_area = gid_area.fillna(0)
gid_area["건물면적"] = gid_area.iloc[:, 1:].sum(axis = 1)

In [54]:
df_100_to_500 = pd.read_csv("./data/df_100_to_500.csv")

In [55]:
df_gid_pop_coef_group_subway_bus_park_acc_build = pd.merge(df_100_to_500, gid_area, on="gid")

In [56]:
df_gid_pop_coef_group_subway_bus_park_acc_build.to_csv("./data/df_100_to_500.csv", index = False)

In [ ]:
# intersection을 위해 buffer slicing 후 geometry로 컬럼명 변경
df_1_area = df_1.loc[:, ["정류장ID", "buffer"]].rename({"buffer":"geometry"}, axis = 1)

# df_1_area를 기준으로 buil_df intersection
bus_with_build_area_df = gpd.overlay(df_1_area, build_df, how = "intersection")




# 100_to_500 기준으로 buil_df intersection
gid_buffer_inter = gpd.overlay(df_gid_crit, build_df, how = "intersection")

# 건물면적 구하기
bus_with_build_area_df["area"] = bus_with_build_area_df.area

# 정류장 ID와 건물분류별 면적 합하기
build_area = bus_with_build_area_df.groupby(["정류장ID", "purpose"])["area"].agg("sum").reset_index()

# pivot table 활용하여 정류장ID별 건물불류 변수 table 생성
table = pd.pivot_table(build_area, values = "area", index = "정류장ID", columns = "purpose").reset_index()

# 결측치 처리
table = table.fillna(0)

# # 500m 버퍼 내의 면적을 구하기 위해 500m 버퍼 면적 구하기(면적)
# denominator = df_1_area[df_1_area["정류장ID"] == '201000266'].area.values[0]

# #  intersection / 500m 면적
# # table.iloc[:, 1:] = table.iloc[:, 1:].apply(lambda x: x / denominator)

# 버퍼 내 전체 건물 면적 구하기
table["건물면적"] = table.iloc[:, 1:].sum(axis = 1)